In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [5]:
import os
import copy
import cv2
import matplotlib.pyplot as plt
import numpy as np
import toml
import glob
import h5py
from tqdm.auto import tqdm
from markovids import vid

In [6]:
from joblib import Parallel, delayed

## User functions

In [7]:
def get_rolling_bground(
    dat_file,
    step_size=1500,
    frame_spacing=20,
    overlap=0,
    agg_func=np.mean,
    reader_kwargs={"threads": 2},
    progress_bar=False,
    save_dir="_bground",
):

    basename = os.path.splitext(os.path.basename(dat_file))[0]
    path = os.path.dirname(dat_file)
    bground_path = os.path.join(path, save_dir, f"{basename}.hdf5")

    os.makedirs(os.path.join(path, save_dir), exist_ok=True)
    
    if os.path.exists(bground_path):
        return None

    vid_reader = vid.io.AutoReader(
        dat_file,
        **reader_kwargs
    )
    steps = range(0, vid_reader.nframes, step_size - overlap)

    bgrounds = []
    idxs = []
    for _step in tqdm(steps, disable=not progress_bar):
        left_edge = _step
        right_edge = left_edge + step_size

        if right_edge > vid_reader.nframes:
            break
        # right_edge = min(left_edge + step_size, reader.nframes)
        use_frames = vid_reader.get_frames(range(left_edge, right_edge, frame_spacing))
        bgrounds.append(agg_func(use_frames, axis=0))
        idxs.append((left_edge + right_edge) / 2) # get midpoint frame index...
    vid_reader.close()

    
    with h5py.File(bground_path, "w") as f:
        f.create_dataset(
            "bground",
            data=np.array(bgrounds),
            compression="gzip",
        )
        f.create_dataset("frame_idxs", data=np.array(idxs), compression="gzip")
    
    return None

# Quantify fluorescence decay over time

In [8]:
# need to use different calibration files for different days...

In [9]:
base_dir = "/mnt/data/jmarkow/active_projects/quantum_dots/"
fluo_files = sorted(glob.glob(os.path.join(base_dir, "**", "Basler*fluorescence.avi"), recursive=True))

In [11]:
delays = []
for _file in fluo_files:
    delays.append(
        delayed(get_rolling_bground)(
            _file,      
        )
    )
print(len(delays))
dat = Parallel(n_jobs=4, verbose=10, backend="multiprocessing")(delays)

1280


[Parallel(n_jobs=4)]: Using backend MultiprocessingBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done   5 tasks      | elapsed:  3.0min
[Parallel(n_jobs=4)]: Done  10 tasks      | elapsed:  4.4min
[Parallel(n_jobs=4)]: Done  17 tasks      | elapsed:  4.6min
[Parallel(n_jobs=4)]: Done  24 tasks      | elapsed:  4.6min
[Parallel(n_jobs=4)]: Done  33 tasks      | elapsed:  4.6min
[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed:  4.6min
[Parallel(n_jobs=4)]: Batch computation too fast (0.16885275519256376s.) Setting batch_size=2.
[Parallel(n_jobs=4)]: Batch computation too fast (0.012085199356079102s.) Setting batch_size=4.
[Parallel(n_jobs=4)]: Done  57 tasks      | elapsed:  4.6min
[Parallel(n_jobs=4)]: Batch computation too fast (0.022510766983032227s.) Setting batch_size=8.
[Parallel(n_jobs=4)]: Done  93 tasks      | elapsed:  4.6min
[Parallel(n_jobs=4)]: Batch computation too fast (0.045400142669677734s.) Setting batch_size=16.
[Parallel(n_jobs=4)]: Batch computation to